## Importing libs and loading the dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
sns.set() # apply the seaborn defaults to plotted figures (e.g. theme, scaling, color palette), instead of matplotlib's

In [7]:
dataset_csv_path = "Metro_Interstate_Traffic_Volume.csv" # 8 attributes of 48,204 examples (plus the target column)

# loading the dataset with pandas (instead of numpy or csv) since there are different datatypes
dataset_csv = pd.read_csv(dataset_csv_path, delimiter=',')
dataset_csv.shape

(48204, 9)

## Dataset attributes
- 1: **holiday**: US National holidays plus regional holiday, Minnesota State Fair
- 2: **temp**: average temp in kelvin
- 3: **rain 1h**: amount in mm of rain that occurred in the hour
- 4: **snow 1h**: amount in mm of snow that occurred in the hour
- 5: **clouds all**: percentage of cloud cover
- 6: **weather main**: short textual description of the current weather
- 7: **weather description**: longer textual description of the current weather
- 8: **date time**: hour of the data collected in local CST time
- target: **traffic volume**: traffic volume

In [8]:
dataset_csv.dtypes

holiday                 object
temp                   float64
rain_1h                float64
snow_1h                float64
clouds_all               int64
weather_main            object
weather_description     object
date_time               object
traffic_volume           int64
dtype: object

## 0. Visualize the data

In [12]:
dataset_csv.agg(["min", "max", "median", "mean", "std"])

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
min,Christmas Day,0.000000,0.000000,0.000000,0.000000,Clear,SQUALLS,2012-10-02 09:00:00,0.000000
max,Washingtons Birthday,310.070000,9831.300000,0.510000,100.000000,Thunderstorm,very heavy rain,2018-09-30 23:00:00,7280.000000
median,NaN,282.450000,0.000000,0.000000,64.000000,NaN,NaN,NaN,3380.000000
mean,NaN,281.205870,0.334264,0.000222,49.362231,NaN,NaN,NaN,3259.818355
std,NaN,13.338232,44.789133,0.008168,39.015750,NaN,NaN,NaN,1986.860670


In [14]:
mean_values = dataset_csv.groupby("weather_main")[["temp", "traffic_volume"]].mean()
mean_values["temp_celsius"] = mean_values["temp"].apply(lambda temp_K: temp_K - 273.15)
mean_values["size"] = dataset_csv.groupby("weather_main").size() # amount of entries in each group

# order by the weather conditions which have the largest mean traffic volume
mean_values[["size", "temp_celsius", "temp", "traffic_volume"]].sort_values(by="traffic_volume", ascending=False)

,size,temp_celsius,temp,traffic_volume
weather_main,,,,
Clouds,15164,8.316101,281.466101,3618.449749
Haze,1360,2.668581,275.818581,3502.101471
Rain,5672,13.829586,286.979586,3317.905501
Drizzle,1821,10.645431,283.795431,3290.727073
Smoke,20,17.777500,290.927500,3237.650000
Clear,13391,8.339971,281.489971,3055.908819
Snow,2876,-5.194812,267.955188,3016.844228
Thunderstorm,1034,19.566431,292.716431,3001.620890
Mist,5950,6.326892,279.476892,2932.956639


In [16]:
# 44737 / 48204 ~= 93.81% of entries have no rain (3467 have)
dataset_csv.groupby("rain_1h").size()

rain_1h
0.00       44737
0.25         948
0.26           2
0.27           5
0.28          23
           ...  
28.70          2
31.75          1
44.45          1
55.63          1
9831.30        1
Length: 372, dtype: int64

In [17]:
# 48141 / 48204 ~= 99.87% of entries have no snow (only 63 have)
dataset_csv.groupby("snow_1h").size()

snow_1h
0.00    48141
0.05       14
0.06       12
0.08        2
0.10        6
0.13        6
0.17        3
0.21        1
0.25        6
0.32        5
0.44        2
0.51        6
dtype: int64

## 1. Clean the dataset

In [28]:
# we'll copy our dataset to the DataFrame object 'data' to clean it and
# save it as a new .csv file later (splitting it into trainning/validation and test)
data = dataset_csv.copy()

### 1.1. Removing invalid data

In [29]:
# removing entries with 0 Kelvin temperature
print(f"{data.shape} - {data.query('temp == 0.0').shape} => ", end='')

data = data.query("temp > 0.0")
print(data.shape) # removes 10 invalid samples

(48204, 9) - (10, 9) => (48194, 9)


In [30]:
# the max value of rain_1h is invalid (>1000mm), so we'll drop it
data.query("rain_1h > 0.0").sort_values(by="rain_1h", ascending=False).head(n=1)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
24872,None,302.11,9831.3,0.0,75,Rain,very heavy rain,2016-07-11 17:00:00,5535


In [31]:
data.drop(index=24872, inplace=True) # removes the row where rain_1h == 9831.30

In [35]:
data.shape

(48193, 9)

### 1.2. Saving the clean dataset

Note that we removed values from the whole dataset, and only then we'll split it into test, train and validation.

However, the values we removed (0 Kelvin temperature and over 1000mm of rain in an hour) are clearly invalid, so even if we're dealing with test (never seen) data, we could confidently remove entries like these (besides that, only 11 rows were actually removed).

In [37]:
data["date_time"] = pd.to_datetime(data["date_time"], format="%Y-%m-%d %H:%M:%S")
data.set_index("date_time", inplace=True)

In [38]:
data.to_csv("clean.csv")

## 2. Split the dataset into train/validation and test

- test: 10%
- train & validation: 90%
    - train: 80%
    - validation: 10%

In [151]:
# random seed for splitting the data deterministically
RANDOM_SEED = 886

In [183]:
from sklearn.model_selection import train_test_split

print(f"total: {data.shape}\n")

train_data, test_validation_data = train_test_split(data, train_size=0.8, shuffle=True, random_state=RANDOM_SEED)
print(f"train: {train_data.shape}") # 80%

validation_data, test_data = train_test_split(test_validation_data, test_size=0.5, shuffle=True, random_state=RANDOM_SEED)
print(f"validation: {validation_data.shape}") # 10% of total
print(f"test: {test_data.shape}") # 10% of total

total: (48193, 8)

train: (38554, 8)
validation: (4819, 8)
test: (4820, 8)


In [184]:
# visualize the first values of each set
print("Train: \n", train_data[["traffic_volume"]].sort_values("date_time").head(n=4))
print("\nValidation: \n", validation_data[["traffic_volume"]].sort_values("date_time").head(n=4))
print("\nTest: \n", test_data[["traffic_volume"]].sort_values("date_time").head(n=4))

Train: 
                      traffic_volume
date_time                          
2012-10-02 09:00:00            5545
2012-10-02 10:00:00            4516
2012-10-02 11:00:00            4767
2012-10-02 12:00:00            5026

Validation: 
                      traffic_volume
date_time                          
2012-10-02 16:00:00            6015
2012-10-03 20:00:00            2898
2012-10-04 09:00:00            5309
2012-10-04 10:00:00            4603

Test: 
                      traffic_volume
date_time                          
2012-10-03 15:00:00            5692
2012-10-03 23:00:00            1015
2012-10-04 03:00:00             367
2012-10-04 07:00:00            6990


## 3. Transform features

### 3.1. Turn 'date_time' into 'hour', 'weekday' and 'weekofyear'

In [185]:
def transformed_date_time(df):
    transformed_df = df.copy()
    transformed_df.insert(0, "hour", pd.DatetimeIndex(transformed_df.index).hour)
    transformed_df.insert(1, "weekday", pd.DatetimeIndex(transformed_df.index).weekday)
    transformed_df.insert(2, "weekofyear", pd.DatetimeIndex(transformed_df.index).weekofyear)
    return transformed_df

### 3.2. Dropping 'snow_1h' and 'weather_description' columns
They both seem redundant if we consider the 'weather_main' values.

In [186]:
def dropped_snow_1h_weather_description(df):
    return df.drop(columns=["snow_1h", "weather_description"])

### 3.3. Adding holiday value to the whole day (not just at 00:00:00)

In [187]:
def transformed_holiday(df):
    transformed_df = df.copy()
    transformed_df["date_time"] = transformed_df.index
    
    holidays = transformed_df[transformed_df.holiday != "None"]
    holiday_names = [row["holiday"] for index, row in holidays.iterrows()]
    holiday_dates = holidays["date_time"].dt.normalize()
    
    # add holiday name to holiday days on hours other than 00:00:00
    for holiday_name, holiday_date in zip(holiday_names, holiday_dates):
        transformed_df.loc[(transformed_df["date_time"].dt.normalize() == holiday_date), "holiday"] = holiday_name
    
    return transformed_df.drop(columns=["date_time"])    

### 3.5. Apply changes to the split sets

In [188]:
def transformed(dataset, transformations=[transformed_date_time, dropped_snow_1h_weather_description, transformed_holiday]):
    for transform in transformations:
        dataset = transform(dataset)
    return dataset

In [189]:
train_data = transformed(train_data)
validation_data = transformed(validation_data)

test_data = transformed(test_data)

### 3.6. Transform categorical features

Instead of turning a categorical feature with $n$ possible values into an $n$-D vector, with entries having only one non-zero — _hot_ — element, we'll drop one dimension (i.e. not use one of the values), creating an $(n-1)$-D vector to avoid the [Dummy Variable Trap](https://www.algosome.com/articles/dummy-variable-trap-regression.html).

In [190]:
# categorical attributes: "holiday", "weather_main"
print(data["holiday"].unique(), '\n')
print(data["weather_main"].unique())

['None' 'Columbus Day' 'Veterans Day' 'Thanksgiving Day' 'Christmas Day'
 'New Years Day' 'Washingtons Birthday' 'Memorial Day' 'Independence Day'
 'State Fair' 'Labor Day' 'Martin Luther King Jr Day'] 

['Clouds' 'Clear' 'Rain' 'Drizzle' 'Mist' 'Haze' 'Fog' 'Thunderstorm'
 'Snow' 'Squall' 'Smoke']


In [191]:
# dummy encoding, n-D: 
#   pd.get_dummies(df, prefix="weather", columns=["weather_main"])

# one-hot encoding, (n-1)-D:
#   pd.get_dummies(df, prefix="weather", columns=["weather_main"], drop_first=True)

def encoded_weather_main(df):
    encoded_df = df.copy()
    weather_encoding = pd.get_dummies(encoded_df["weather_main"], drop_first=True) # drops Clear weather
    encoded_df = pd.concat([encoded_df, weather_encoding], axis=1)
    encoded_df.drop(columns=["weather_main"], inplace=True)
    return encoded_df

In [192]:
# we'll make 'holiday' a binary value
def encoded_holiday(df):
    encoded_df = df.copy()
    # obs.: comparing the value to 0 prevents accidents when running this twice
    encoded_df["holiday"] = encoded_df["holiday"].apply(lambda h: 0 if (h == "None" or h == 0) else 1)
    return encoded_df

In [193]:
# trasformed categorical features
def encoded(dataset, encodings=[encoded_weather_main, encoded_holiday]):
    for encode in encodings:
        dataset = encode(dataset)
    return dataset

In [194]:
train_data = encoded(train_data)
validation_data = encoded(validation_data)

test_data = encoded(test_data)

In [195]:
# make 'traffic_volume' the last column
columns_order = lambda df: [col for col in df.columns.values if col != "traffic_volume"] + ["traffic_volume"]

train_data = train_data[columns_order(train_data)]
validation_data = validation_data[columns_order(validation_data)]

test_data = test_data[columns_order(test_data)]

## 4. Normalize values

It's important we **normalize our trainning data** and save the mapping we made to it so we can use the same values once we deal with the validation and test sets (i.e. we don't want to be normalizing the test and validation sets based on their own values).

In [199]:
# NOTE: we have to change this list if new columns are added
attrs_to_normalize = ["traffic_volume", "hour", "weekday", "weekofyear", "temp", "rain_1h", "clouds_all"]
def get_stats(df, columns, operations=['min', 'max', 'median', 'mean', 'std']):
    return df[columns].agg(operations)

Z-score normalization: $ x \leftarrow \dfrac{x - \mu}{\sigma}$ (makes the mean 0 and the standard deviation 1)

In [200]:
# mu = mean, sigma = stddev
def z_score(x, mu, sigma):
    return (x - mu) / sigma

def normalized(dataset, stats):
    normalized_dataset = dataset.copy()
    for col in stats:
        col_mean = stats.loc["mean", col]
        col_stddev = stats.loc["std", col]
        normalized_dataset.loc[:, col] = dataset.loc[:, col].apply(lambda x: z_score(x, mu=col_mean, sigma=col_stddev))
    return normalized_dataset

In [201]:
# NOTE: we must use 'train_stats' when normalizing 'validation_data' and 'test_data' aswell
train_stats = get_stats(train_data, columns=attrs_to_normalize).copy()

In [202]:
train_stats

,traffic_volume,hour,weekday,weekofyear,temp,rain_1h,clouds_all
min,0.000000,0.000000,0.000000,1.000000,243.390000,0.000000,0.000000
max,7280.000000,23.000000,6.000000,53.000000,310.070000,55.630000,100.000000
median,3370.000000,11.000000,3.000000,27.000000,282.465000,0.000000,64.000000
mean,3252.085594,11.372750,2.993230,26.467863,281.246050,0.131921,49.442289
std,1987.407287,6.944758,2.004335,14.850605,12.728783,1.025035,38.990004


In [203]:
train_data = normalized(train_data, stats=train_stats)
validation_data = normalized(validation_data, stats=train_stats)
test_data = normalized(test_data, stats=train_stats)

In [204]:
# we should now see mean~=0 and std~=1 for all attributes
get_stats(train_data, columns=attrs_to_normalize)

,traffic_volume,hour,weekday,weekofyear,temp,rain_1h,clouds_all
min,-1.636346e+00,-1.637602e+00,-1.493378e+00,-1.714938e+00,-2.974051e+00,-1.286994e-01,-1.268076e+00
max,2.026718e+00,1.674248e+00,1.500133e+00,1.786603e+00,2.264470e+00,5.414260e+01,1.296684e+00
median,5.933077e-02,-5.367356e-02,3.377542e-03,3.583266e-02,9.576331e-02,-1.286994e-01,3.733704e-01
mean,5.667165e-17,2.543313e-17,-3.953193e-17,-5.897538e-18,9.187258e-17,4.791750e-18,3.133067e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [205]:
# we should now see mean close to 0 and std close to 1
get_stats(validation_data, columns=attrs_to_normalize)

,traffic_volume,hour,weekday,weekofyear,temp,rain_1h,clouds_all
min,-1.634836,-1.637602,-1.493378,-1.714938,-2.908845,-0.128699,-1.268076
max,1.906964,1.674248,1.500133,1.786603,2.170196,24.572889,1.296684
median,0.129271,0.090320,0.003378,0.035833,0.115797,-0.128699,0.373370
mean,0.031358,0.021147,0.001514,0.024347,0.013055,0.003504,-0.008086
std,0.994448,0.985955,1.007676,0.997670,0.995840,0.901125,1.003678


In [206]:
# we should now see mean close to 0 and std close to 1
get_stats(test_data, columns=attrs_to_normalize)

,traffic_volume,hour,weekday,weekofyear,temp,rain_1h,clouds_all
min,-1.635843,-1.637602,-1.493378,-1.714938,-2.861707,-0.128699,-1.268076
max,2.016655,1.674248,1.500133,1.786603,2.004430,24.709470,1.296684
median,0.043984,-0.053674,0.003378,0.035833,0.068620,-0.128699,0.373370
mean,0.009341,0.017158,-0.044651,-0.008174,0.000880,-0.018913,-0.009955
std,1.002002,1.007484,0.999010,0.995686,0.988977,0.878543,1.002495


## 5. Save the split data, normalized with values from the training set

In [207]:
train_data.to_csv("train_80.csv")
validation_data.to_csv("validation_10.csv")

test_data.to_csv("test_10.csv") # and now.. don't touch it anymore!